## <a href="https://cursos.alura.com.br/course/langchain-python-ferramentas-llm-openai/task/156163"><b>Implementando uma SimpleSequentialChain para a criação de um roteiro de viagens</b><br/>(Técnica de Engenharia de Prompts Successive Prompts)</a><br/>
##### <li> O objetivo será criar uma cadeia de chamadas, cada uma delas representando um prompt</li> 

#### <b>PASSO 1 - CRIANDO O PROMPT TEMPLATE</b> (Pré requisito aula passada) (<a href="https://drive.google.com/file/d/1mdX8hLi2KkOaIBMIrB5Y00wKmtbTuwE_/view?usp=sharing">Pré requisito aula passada</a>)

In [5]:
from langchain_openai import ChatOpenAI # JÁ PEDE A API DE CHAT DIRETO
from os import getenv
from dotenv import load_dotenv # CARREGA A VARIÁVEL DE AMBIENTE OPENAI_KEY LIDA DO ARQUIVO .env
from langchain.prompts import PromptTemplate

load_dotenv() # CARREGANDO O ARQUIVO COM A OPENAI_KEY

numero_de_dias = 7
numero_de_criancas = 2
atividade = "praia"

# PASSO 1 - CRIANDO O TEMPLATE
template = "Crie um roteiro de viagem de {dias} dias, para uma família com {criancas} crianças, que gostam de {atividade} e cite a cidade." # TEMPLATE É UM MODELO DE FORMULÁRIO
                                                                                                                                            # PERCEBA QUE NÃO ESTÁ COM O f NO INÍCIO, 
                                                                                                                                            # PARA PEGAR VALORES DE VARIÁVIES
# PASSO 2 - INSTANCIANDO E INICIALIZANDO O PROMPT TEMPLATE A PARTIR DO TEMPLATE (from_template)
prompt_template = PromptTemplate.from_template(template) # INICIALIZANDO A PARTIR DE UMA STRING, PODE-SE INICIALIZAR A PARTIR DE OUTRAS FORMAS (Ex: Arquivo)

# PASSO 3 - FORMATANDO O TEMPLATE
# FORMATANDO O prompt_template. PRENCHENDO O MEU FORMULÁRIO
prompt_template = prompt_template.format(dias=numero_de_dias, criancas=numero_de_criancas, atividade=atividade)

# PROMPT CRIADO
prompt = prompt_template

print(prompt)

Crie um roteiro de viagem de 7 dias, para uma família com 2 crianças, que gostam de praia e cite a cidade.


In [4]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### <b>PASSO 2 - CRIANDO O CHAT PROMPT TEMPLATE</b></br>

<ul><li>Também possui várias formas de inicializar (Métodos from_), assim como o PromptTemplate (<a href="https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html">Ver na documentação</a>)

In [ ]:
from langchain import ChatPromptTemplate # Uma interação de múltiplos prompts, é um chat, por isso, vamos importar um ChatPromptTemplate




Crie um roteiro de viagem de 7 dias, para uma família com 2 crianças, que gostam de praia e cite a cidade.


#### <b>PASSO 3 - INSTANCIANDO O LLM</b></br>

In [ ]:
llm = ChatOpenAI( # INSTANCIANDO A LLM
                    model="gpt-4.1-mini",
                    temperature=0.5,
                    # 1 - OBTENDO A API KEY POR MEIO DA VARIÁVEL DE AMBIENTE OPENAI_KEY. QUE VAI FICAR ARMAZENADA NO ARQUIVO .env.
                    # 2 - AINDA É NECESSÁRIO CARREGAR ESSE ARQUIVO. VER NA PRIMEIRA CÉLULA DO NOTEBOOK
                    api_key=getenv("OPENAI_KEY")
                )

#### <b>PASSO 4 - INVOCANDO</b>

In [21]:
# SE EU QUISER UTILIZAR A MINHA LLM, BASTA INVOCAR ALGO NELA
resposta = llm.invoke(prompt)
print(resposta)

content='Claro! Vou criar um roteiro de 7 dias para uma família com duas crianças que gostam de praia, na cidade de Florianópolis, Santa Catarina, que é conhecida por suas praias lindas e infraestrutura familiar.\n\n---\n\n### Roteiro de 7 dias em Florianópolis para família com crianças\n\n**Dia 1: Chegada e Praia da Joaquina**  \n- Chegada em Florianópolis e acomodação no hotel ou pousada.  \n- Tarde na Praia da Joaquina, famosa pelas dunas e ondas, mas também com áreas calmas para as crianças brincarem na areia.  \n- Aproveite para almoçar em um dos quiosques à beira-mar.  \n\n**Dia 2: Praia dos Ingleses**  \n- Manhã na Praia dos Ingleses, que tem águas calmas e é ótima para crianças.  \n- Almoço em restaurante familiar na região.  \n- À tarde, passeio pelo centrinho dos Ingleses para sorvete e compras de artesanato local.  \n\n**Dia 3: Projeto Tamar e Praia do Forte**  \n- Visita ao Projeto Tamar, que fica na Praia do Forte, para conhecer as tartarugas marinhas e aprender sobre a co

#### <b>OBTENDO APENAS O CONTEÚDO DA MENSAGEM</b>

In [ ]:
print(resposta.content)